In [2]:
from src.stepper import StepperMotor
import logging

# Configure debug logging
logging.basicConfig(
    level=logging.DEBUG, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


with StepperMotor("COM5") as motor:
    motor.enable()

    print(f"Version: {motor.get_version()}")
    print(f"Position: {motor.get_position()}")
    print(f"Position error: {motor.get_position_error()}")

    print("Drive config")
    for items in motor.get_drive_config().items():
        print(items)

    print(f"Status: {motor.get_status()}")
    print(f"System status: {motor.get_system_status()}")
    print(f"Homing status: {motor.get_homing_status()}")
    print(f"Motor status: {motor.get_encoder_calibration_status()}")

    print(f"Bus voltage: {motor.get_bus_voltage()}")
    print(f"Phase current: {motor.get_phase_current()}")


2024-11-26 16:00:52,154 - StepperMotor - DEBUG - Command: Enable motor (store=0, sync=0)
TX: 01 F3 AB 01 00 6B
2024-11-26 16:00:52,158 - StepperMotor - DEBUG - Response to Enable motor (store=0, sync=0):
RX: 01 F3 02 6B
2024-11-26 16:00:52,158 - StepperMotor - DEBUG - Command: Read firmware version
TX: 01 1F 6B
2024-11-26 16:00:52,162 - StepperMotor - DEBUG - Response to Read firmware version:
RX: 01 1F FF 78 6B
2024-11-26 16:00:52,162 - StepperMotor - DEBUG - Command: Read current position
TX: 01 36 6B
2024-11-26 16:00:52,165 - StepperMotor - DEBUG - Response to Read current position:
RX: 01 36 01 00 1C FB A5 6B
2024-11-26 16:00:52,165 - StepperMotor - DEBUG - Command: Read position error
TX: 01 37 6B
2024-11-26 16:00:52,168 - StepperMotor - DEBUG - Response to Read position error:
RX: 01 37 01 00 00 00 02 6B
2024-11-26 16:00:52,169 - StepperMotor - DEBUG - Command: Read drive configuration
TX: 01 42 6C 6B
2024-11-26 16:00:52,175 - StepperMotor - DEBUG - Response to Read drive configu

Version: x
Position: Position(steps=1899429, revolutions=28, degrees=353.9)
Position error: 2
Drive config
('byte_count', 33)
('param_count', 21)
('motor_type', '1.8°')
('pulse_control_mode', 2)
('communication_mode', 2)
('en_pin_level', 2)
('dir_pin_direction', 'CW')
('subdivision', 16)
('subdivision_interpolation', True)
('auto_screen_off', False)
('open_loop_current', 1000)
('max_closed_loop_current', 3000)
('max_output_voltage', 5000)
('uart_baud_rate', 115200)
('can_rate', 500000)
('device_id', 1)
('checksum', 0)
('command_response_mode', 'Acknowledge')
('stall_protection', True)
('stall_speed_threshold', 8)
('stall_current_threshold', 2600)
('stall_detection_time', 2000)
('position_arrival_window', 0.3)
Status: MotorStatus(enabled=True, position_reached=True, stalled=False, protection_triggered=False)
System status: {'bus_voltage': 2.35, 'phase_current': 43.776, 'encoder_value': 5892, 'position': 1073807388, 'speed': -1113, 'status_flags': 0}
Homing status: {'encoder_ready': True

In [11]:
import time
for i in range(100):
    with StepperMotor("COM5") as motor:
        motor.enable()
        motor.set_position(steps=80, speed=3000, acceleration=255)
    time.sleep(0.5)


2024-11-26 16:05:43,921 - StepperMotor - DEBUG - Command: Enable motor (store=0, sync=0)
TX: 01 F3 AB 01 00 6B
2024-11-26 16:05:43,925 - StepperMotor - DEBUG - Response to Enable motor (store=0, sync=0):
RX: 01 F3 02 6B
2024-11-26 16:05:43,925 - StepperMotor - DEBUG - Command: Move relative 80 steps at 3000 RPM (dir=1, accel=255)
TX: 01 FD 01 0B B8 FF 00 00 00 50 00 00 6B
2024-11-26 16:05:43,929 - StepperMotor - DEBUG - Response to Move relative 80 steps at 3000 RPM (dir=1, accel=255):
RX: 01 FD 02 6B
2024-11-26 16:05:44,467 - StepperMotor - DEBUG - Command: Enable motor (store=0, sync=0)
TX: 01 F3 AB 01 00 6B
2024-11-26 16:05:44,471 - StepperMotor - DEBUG - Response to Enable motor (store=0, sync=0):
RX: 01 F3 02 6B
2024-11-26 16:05:44,471 - StepperMotor - DEBUG - Command: Move relative 80 steps at 3000 RPM (dir=1, accel=255)
TX: 01 FD 01 0B B8 FF 00 00 00 50 00 00 6B
2024-11-26 16:05:44,475 - StepperMotor - DEBUG - Response to Move relative 80 steps at 3000 RPM (dir=1, accel=255):
RX

In [ ]:

with StepperMotor("COM5") as motor:
    try:
        motor.enable()
        logger.debug("Motor enabled successfully")
                
    except Exception as e:
        logger.error(f"Communication error: {str(e)}", exc_info=True)
        status = motor.get_status()

    try:
        motor.home(mode=0)  # Home to nearest position
    except Exception as e:
        logger.error(f"Homing failed: {str(e)}")
        raise
    motor.set_zero()    # Set current position as zero
    
    # Move with acceleration
    motor.set_position(
        steps=32000,    # 10 revolutions
        speed=1500,     # 1500 RPM
        acceleration=10  # Smooth acceleration
    )
    
    # Monitor status
    status = motor.get_status()
    if status.stalled:
        motor.clear_stall()
        motor.emergency_stop()
